In [109]:
import config
import pandas as pd
import json

In [113]:
import re
import os
import time
import random
import requests
import numpy as np
import pandas as pd
from os import system
from math import floor
from copy import deepcopy
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# BreweryDB

In [13]:
def get_brewery_info(url, api_key, page, page_limit):
    """function to pull the information from the Brewery DB API"""
    paged_data = []
    while page <= page_limit:
        url_params = {'key': api_key, 'format':'json', 'p': page, 'withIngredients': 'Y'}
        response = requests.get(url, params=url_params)
        paged_data.append(response.json())
        page += 1
    return paged_data

In [14]:
api_key = config.key
# url_params = {'key': api_key, 'format':'json', 'p': page, 'withBreweries': 'Y', 'withIngredients': 'Y'}
url = "https://sandbox-api.brewerydb.com/v2/beers/"

In [26]:
data = get_brewery_info(url, api_key, 1, 50)

In [13]:
data[22].keys()

dict_keys(['currentPage', 'numberOfPages', 'totalResults', 'data', 'status'])

In [27]:
data[1]['data'][3]['name']

'American Summer Hoppy Wit'

In [47]:
#for now I just want the name
def get_names(data):
    list_names = []
    for page in data:
        for item in page['data']:
            print(item['name'])
#             list_names.append(item['name'])
    return list_names

In [49]:
list_names = get_names(data)

KeyError: 'data'

In [50]:
list_names

["'Murican Pilsner",
 '11.5° PLATO',
 '12th Of Never',
 '15th Anniversary Ale',
 '16 So Fine Red Wheat Wine',
 '1794 The Fergal Project',
 '17th Saison',
 '18th Anniversary Belgian Tripel',
 '19 - Golden Belgian Style Ale',
 '1904 American Red Lager',
 '2 x 4',
 '200th Anniversary Export Stout',
 '2017 Beer Camp',
 '20th Anniversary Imperial Hash IPA on Brett',
 '20th Street Ale Citra',
 '20th Street Ale Crystal',
 '20th Street Ale Magnum',
 '21st Anniversary',
 '25th Anniversary',
 '3 Weight',
 "30th Anniversary - Charlie, Fred & Ken's Bock",
 "30th Anniversary - Fritz and Ken's Ale",
 '30th Anniversary - Grand Cru',
 "30th Anniversary - Jack & Ken's Ale",
 "35th Anniversary  - Brewer's Reserve",
 '420 Extra Pale Ale',
 '420 Fest',
 '420 Strain G13 IPA',
 '471 Double IPA - Hull Melon',
 '471 ESB - Extra Special Bitter',
 '471 IPA Barrel Series: Citra',
 '471 IPA Barrel Series: Eureka!',
 '471 IPA. Aggressive Hoppiness',
 '471 Pilsner',
 '471 Small Batch IPA',
 "5 C's IPA",
 '7 Birds',

In [17]:
def parse(data_lst):
    # elements of the style keys list to look for when we parse through style 
    style_lst = ["id", "name", "shortName",'abvMax','srmMin', 'srmMax', "ibuMax", "ibuMin",
                 'ogMin','fgMin','fgMax' ]
    api_dict_list = []
    # read_through each in the data_list
    for index in range(0,23):
        print(index)
        for api_call in data_lst[index]["data"]:
            api_dict = {}
            api_dict["name"] = api_call["name"]
            if 'abv' in api_call:
                api_dict["abv"] = float(api_call["abv"])
            if 'ibu' in api_call:
                api_dict['ibu'] = float(api_call["ibu"])
            if api_call['isOrganic'] == 'Y':
                api_dict["is_organic"] = 1
            else:
                api_dict['is_organic'] = 0
            api_dict["created"] = api_call["createDate"]
            if "style" in api_call:
                for style in style_lst:
                    if style in api_call["style"]:
                        api_dict[f"style_{style}"] = api_call["style"][style]
                api_dict["category_name"] = api_call["style"]["category"]["name"]
                api_dict["category_id"] = api_call["style"]["category"]["id"]
            brewery = api_call["breweries"][0]
            api_dict[f"brewery_name"] =  brewery["name"]
            api_dict[f"brewery_id"] = brewery["id"]
            if "established" in api_call["breweries"]:
                api_dict[f"established"] = brewery["established"]
            api_dict['location'] = brewery["locations"][0]['region']
#             api_dict["brewery_locations_map"] = []
#             api_dict["brewery_locations"] = []
#             api_dict["brewery_locality"] = ""
#             count = 1
#             for idx in range(len(brewery["locations"])):
#                 brew_location = brewery["locations"][0]
#                 api_dict["brewery_locations"].append(brew_location["name"])
#                 if "latitude" in brew_location and "longitude" in brew_location:
#                     api_dict["brewery_locations_map"].append([float(brew_location["latitude"]),
#                                                              float(brew_location["longitude"])])
#                 if "locality" in brew_location:
#                     api_dict["brewery_locality"]+= f'{count}. {brew_location["locality"]}' + " "
#                     count += 1
#             api_dict["number_of_locations"] = len(api_dict["brewery_locations"])
            api_dict_list.append(api_dict)
    return api_dict_list

# Beer Advocate

In [63]:
driver = webdriver.Chrome()

In [64]:
driver.get('https://www.beeradvocate.com/community/login/')

In [67]:
driver.find_element_by_xpath('//*[@id="navigation"]/div/div/div/div/nav/div/ul[1]/li[5]/a[1]').click()

In [69]:
comment = driver.find_element_by_xpath('//*[@id="rating_fullview_content_2"]/div[1]').text

In [70]:
comment

'"Garden Party is a blend of sour blond and wheat ales aged in white wine barrels for up to two years with Willamette Valley-grown chardonnay grapes followed by a late infusion of fresh Meyer lemon zest and dried elderflowers. The perfect beverage for any outdoor soirée, Garden Party delivers the sparkle of champagne, the floral delicacy of elderflowers and the brilliance of sweet lemon zest—bringing refinement and refreshment to your party, wherever it may be."\n\n5 oz pour on tap. The first sip yields heavy funk and a surprising amount of sweetness - not a great combination. However, the sweetness mellows out and gives way to oak, lemon, vanilla, white grape juice, and a floral component that must be the elderflowers. A unique beer for sure and decently complex. Not my favorite, but better than it seemed at first sip.'

In [71]:
atts = driver.find_element_by_xpath('//*[@id="rating_fullview_content_2"]/span[5]').text

In [72]:
atts

'look: 3.75 | smell: 4 | taste: 4.25 | feel: 4 | overall: 4'

## straight to beer url:

In [77]:
name = driver.find_element_by_xpath('//*[@id="content"]/div/div/div[2]/div/div/div[1]/h1').text

In [81]:
name.split('\n')[1]

'Sierra Nevada Brewing Co.'

In [75]:
img = driver.find_element_by_xpath('//*[@id="main_pic_mobile"]/div/img')
src = img.get_attribute('src')

In [76]:
src

'https://cdn.beeradvocate.com/im/beers/276.jpg'

In [ ]:
beer = {}
beer['name'] = driver.find_element_by_xpath('//*[@id="content"]/div/div/div[2]/div/div/div[1]/h1').text


In [99]:
listy = driver.find_elements_by_xpath('//*[@id="rating_fullview_content_2"]')

In [100]:
for item in listy:
    print(item.text)

4/5  rDev -1.2%
look: 4 | smell: 4 | taste: 4 | feel: 4 | overall: 4

11/9/19 - Single from Elkton; did a comparison with Monument City Pagoda (pale ale). Both are very nice. Monument is a bit sharper compared to the milder SNPA, but very nice as well.

 182 characters

DuaneL, Nov 09, 2019  Report
4.71/5  rDev +16.3%
look: 4.5 | smell: 4.75 | taste: 4.75 | feel: 4.5 | overall: 4.75

An exceptional pale ale by any standard. Golden amber in colour, pours well with nice head. Really enjoyable citrus aroma and flavour that lingers but isn't bitter at the end like others I've tried. Smooth and drinkable. Overall one of my favourites.

 254 characters

MMoore1971, Nov 05, 2019  Report
3.83/5  rDev -5.4%
look: 3.75 | smell: 3.75 | taste: 4 | feel: 3.5 | overall: 3.75

Pours golden with a 1/2 inch white head. Moderate lacing as you go.

Aroma is grapefruity, piney, resiny, grassy.

Tastes of strong pine and grapefruit. Good bitterness.

Medium bodied, dry mouthfeel. Light carbonation.

While 

4.24/5  rDev +4.7%
look: 4.25 | smell: 4 | taste: 4.25 | feel: 4.25 | overall: 4.5

12oz bottle - dated 03/05/19 - poured into a Spiegelau-style IPA glass at 45F

Pours a bright copper hue, nearly crystal clear, with about two fingers of very off white head, which possesses impressive retention, never fully dissipating to less than a full cap, and leaving a perfect ring of clingy lace behind. Fine ascending streams of effervescence are visible.

Nose: A hefty base of caramel malt is apparent on the nose. Slightly sweet, slightly toasty, crisp, and warm. Among the malt, a generous dose of Cascade hops waft up, with aromas of grapefruit, lemon, pine resin, pear, and perhaps even a note of sweet ripe mango. There is no earthiness, but rather a green element to the nose that evokes images of a forest. Lots of floral notes as well, including lilac, hyacinth, and Easter lilly.

Palate: Like the nose, the first sip unfolds with caramel malt. Semi-sweet, slightly toasty, and crisp, with notes 

In [105]:
listy = driver.find_elements_by_class_name('user-comment')

In [107]:
listy_text = []
for item in listy:
    listy_text.append(item.text)

In [108]:
listy_text

['4/5  rDev -1.2%\nlook: 4 | smell: 4 | taste: 4 | feel: 4 | overall: 4\n\n11/9/19 - Single from Elkton; did a comparison with Monument City Pagoda (pale ale). Both are very nice. Monument is a bit sharper compared to the milder SNPA, but very nice as well.\n\n 182 characters\n\nDuaneL, Nov 09, 2019  Report',
 "4.71/5  rDev +16.3%\nlook: 4.5 | smell: 4.75 | taste: 4.75 | feel: 4.5 | overall: 4.75\n\nAn exceptional pale ale by any standard. Golden amber in colour, pours well with nice head. Really enjoyable citrus aroma and flavour that lingers but isn't bitter at the end like others I've tried. Smooth and drinkable. Overall one of my favourites.\n\n 254 characters\n\nMMoore1971, Nov 05, 2019  Report",
 "3.83/5  rDev -5.4%\nlook: 3.75 | smell: 3.75 | taste: 4 | feel: 3.5 | overall: 3.75\n\nPours golden with a 1/2 inch white head. Moderate lacing as you go.\n\nAroma is grapefruity, piney, resiny, grassy.\n\nTastes of strong pine and grapefruit. Good bitterness.\n\nMedium bodied, dry mout

In [52]:
def geturl(beer):
    driver.get('https://www.beeradvocate.com/beer/')
    time.sleep(.5)
    search_butt = driver.find_element_by_xpath('//*[@id="QuickSearchPlaceholder"]')
    search_butt.click()
    search_beer_name=driver.find_element_by_xpath('//*[@id="QuickSearchQuery"]')
    search_beer_name.send_keys(beer)
    click_search=driver.find_element_by_xpath('//*[@id="QuickSearch"]/form/div[2]/dl/dd/input').click()
    url=driver.current_url
    if url.split('/')[3]=='beer':
        return driver.current_url
    else: 
        try:
            driver.find_element_by_xpath('//*[@id="ba-content"]/div[1]/span/li').text.split('.')[0]=='No results'
            return None
        except:
            
            driver.find_element_by_xpath('//*[@id="ba-content"]/div[2]/div/a[1]/span').click()
            url=driver.current_url
            return url

In [5]:
geturl('sierra nevada pale ale')

'https://www.beeradvocate.com/beer/profile/140/276/'

In [9]:
def get_beer_info(list_urls):
    list_info = []
    for url in list_urls:
        print(url)
        page = requests.get(url)
        soup = BS(page.content, 'html.parser')
        data = soup.findAll(class_ = 'beerstats')
        rating = soup.find(class_ = 'ba-ravg Tooltip').get_text()
        if not soup.find(class_ = 'ba-score Tooltip'):
            score = None
        else:
            score = soup.find(class_ = 'ba-score Tooltip').get_text()
        no_reviews = soup.find(class_ = 'ba-reviews Tooltip').get_text()
        no_ratings = soup.find(class_ = 'ba-ratings Tooltip').get_text()
        if len(soup.findAll(title="Percentage of alcohol by volume.")) == 1:
            abv = None
        else:
            abv = soup.findAll(title="Percentage of alcohol by volume.")[1].get_text()
        avail = soup.findAll(title="The beer's availability.")[1].get_text()
        style = soup.find(title="Learn more about this style.").get_text()
        state = [d.get_text().split(',')[0] for d in data if d.get_text().split(',')[0] in states]
        if len(state) == 0:
            place = 'World'
        else:
            place = state[0]
        beer_info = (rating, score, no_reviews, no_ratings, abv, avail, style, place)
        list_info.append(beer_info)
    return list_info

In [11]:
def do_everything(list_beer_names):
    urls = []
    for beer in list_beer_names:
        urls.append(geturl(beer))
    return urls

In [54]:
list_urls = do_everything(list_names[1])

KeyboardInterrupt: 

In [55]:
list_names[1]

'11.5° PLATO'

In [62]:
driver.quit()

# BS to get urls

In [124]:
url = 'https://www.beeradvocate.com/beer/styles/'
page = requests.get(url)
soup = BS(page.content, 'html.parser')

In [126]:
url = 'https://www.beeradvocate.com/beer/styles/'
page = requests.get(url)
data = page.text
soup = BS(data, 'html.parser')

In [128]:
soup

<!DOCTYPE html>

<html class="Public NoJs uix_javascriptNeedsInit LoggedOut Sidebar Responsive pageIsLtr not_hasTabLinks hasSearch is-sidebarOpen hasRightSidebar is-setWidth navStyle_0 pageStyle_0 hasFlexbox" dir="LTR" id="XenForo" lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<base href="https://www.beeradvocate.com/community/"/>
<script>
			var _b = document.getElementsByTagName('base')[0], _bH = "https://www.beeradvocate.com/community/";
			if (_b && _b.href != _bH) _b.href = _bH;
		</script>
<title>Beer Styles | BeerAdvocate</title>
<noscript><style>.JsOnly, .jsOnly { display: none !important; }</style></noscript>
<link href="css.php?css=xenforo,form,public&amp;style=6&amp;dir=LTR&amp;d=1574286681" rel="stylesheet"/>
<link href="css.php?css=login_bar,moderator_bar,uix_extendedFooter&amp;style=6&amp;dir=LTR&

In [119]:
tags = soup.find_all('a')

In [120]:
for tag in tags:
    if
    print(tag.get('href'))

#
/
/
/about/
/advertise/
/contact/
https://www.beeradvocate.com/community/
https://www.beeradvocate.com/community/
search/?type=post
find-new/posts
/beer/
/beer/
/user/beers/
/forms/add-beer/
/beer/styles/
/beer/new/
/trading/
/beer/top-rated/
/beer/top-rated/
/beer/top-rated/
/beer/top-rated/us/
/beer/top-styles/
/beer/trending/
/beer/top-new/
/beer/fame/
/beer/popular/
/place/
/place/
/user/places/
/toolbox/add_place/
/place/directory/
/place/nearby/
/society/
/community/threads/frequently-asked-questions-faq.444392/
/community/threads/frequently-asked-questions-faq.444392/
/about/
/contact/
/community/forums/help-resources.32/
/community/forums/beeradvocate-talk.34/
/community/threads/how-to-claim-your-place.553982/
#
lost-password/
#
https://www.beeradvocate.com
https://www.beeradvocate.com
#
/
/
/about/
/advertise/
/contact/
https://www.beeradvocate.com/community/
https://www.beeradvocate.com/community/
search/?type=post
find-new/posts
/beer/
/beer/
/user/beers/
/forms/add-beer/


In [ ]:
list_beer_style_urls = ['/beer/styles/32/','/beer/styles/35/','/beer/styles/36/','/beer/styles/33/',
                        '/beer/styles/92/','/beer/styles/73/','/beer/styles/74/','/beer/styles/75/',
                        '/beer/styles/86/','/beer/styles/175/','/beer/styles/119/','/beer/styles/57/',
                        '/beer/styles/18/','/beer/styles/68/','/beer/styles/47/','/beer/styles/147/',
                        '/beer/styles/149/','/beer/styles/29/','/beer/styles/7/','/beer/styles/31/',
                        '/beer/styles/46/','/beer/styles/30/','/beer/styles/6/','/beer/styles/141/',
                        '/beer/styles/114/',
'/beer/styles/132/'
'/beer/styles/199/'
'/beer/styles/140/'
'/beer/styles/116/'
'/beer/styles/174/'
'/beer/styles/150/'
'/beer/styles/189/'
'/beer/styles/128/'
'/beer/styles/99/'
'/beer/styles/97/'
'/beer/styles/190/'
'/beer/styles/54/'
'/beer/styles/129/'
'/beer/styles/98/'
'/beer/styles/66/'
'/beer/styles/154/'
'/beer/styles/76/'
'/beer/styles/127/'
'/beer/styles/85/'
'/beer/styles/161/'
'/beer/styles/38/'
'/beer/styles/164/'
'/beer/styles/155/'
'/beer/styles/39/'
'/beer/styles/42/'
'/beer/styles/40/'
'/beer/styles/20/'
'/beer/styles/37/'
'/beer/styles/43/'
'/beer/styles/21/'
'/beer/styles/131/'
'/beer/styles/41/'
'/beer/styles/192/'
'/beer/styles/159/'
'/beer/styles/80/'
'/beer/styles/101/'
'/beer/styles/193/'
'/beer/styles/194/'
'/beer/styles/163/'
'/beer/styles/148/'
'/beer/styles/9/'
'/beer/styles/8/'
'/beer/styles/168/'
'/beer/styles/169/'
'/beer/styles/5/'
'/beer/styles/72/'
'/beer/styles/173/'
'/beer/styles/12/'
'/beer/styles/70/'
'/beer/styles/11/'
'/beer/styles/157/'
'/beer/styles/158/'
'/beer/styles/69/'
'/beer/styles/13/'
'/beer/styles/82/'
'/beer/styles/95/'
'/beer/styles/162/'
'/beer/styles/84/'
'/beer/styles/19/'
'/beer/styles/191/'
'/beer/styles/78/'
'/beer/styles/60/'
'/beer/styles/142/'
'/beer/styles/56/'
'/beer/styles/55/'
'/beer/styles/58/'
'/beer/styles/152/'
'/beer/styles/79/'
'/beer/styles/165/'
'/beer/styles/77/'
'/beer/styles/94/'
'/beer/styles/93/'
'/beer/styles/48/'
'/beer/styles/87/'
'/beer/styles/91/'
'/beer/styles/89/'
'/beer/styles/90/'
'/beer/styles/198/'
'/beer/styles/171/'
'/beer/styles/15/'
'/beer/styles/10/'
'/beer/styles/14/'
'/beer/styles/50/'
'/beer/styles/52/'
'/beer/styles/53/'
'/beer/styles/16/']